<img src="media/Cryo+ESA_Logo_plain-50gray.svg" alt="CCI Cryosphere banner" width="1000"/>

# Cryosat-2 Sea ice thickness maps with Cate  and the Zarr Data Store
In Cate's Zarr Data Store you can find one [sea ice thickness product derived from Cryosat-2](https://climate.esa.int/en/odp/#/project/sea-ice). This exercise will show you how to visualize and process it.

## Preparations
If you haven't done so please follow the [Cate tutorial](futurelearn.com/tbd) to get started using the exercises.


## Querying the opendata portal for ice thickness products

In [ ]:
# To get things started we need to initialize a few things
#Load some python modules to make them accessible to the notebook
from cate.core.ds import DATA_STORE_POOL
import cate.ops as ops
from cate.util.monitor import ConsoleMonitor
from cate.core.ds import get_metadata_from_descriptor
from cate.ops.io import open_dataset

# the following is needed to run Cate in a Jupyter Notebook
from xcube.util.ipython import enable_asyncio
enable_asyncio()

# utilities
from IPython.display import display
import numpy as np
from datetime import datetime

monitor=ConsoleMonitor()

To begin, let us see which data stores are available in the Data Store Pool.

In [ ]:
DATA_STORE_POOL.store_instance_ids

We see three stores. The 'cci-store' is a store that provides access to all datasets from the CCI Open Data Portal. The 'cci-zarr-store' is a store that contains selected data from the Open Data Portal, converted to the zarr format. The datasets from this store can be opened and processed faster, but the store provides only a small subset of what is offered by the 'cci-store'. The 'local' data store finally allows to access locally provided data. Also, when you select to cache data, you will find it in this store. Cached data can also be opened quickly.

For this exercise we use the 'cci-zarr-store'. As this datat store only holds a few datasets, it is fine to list its content entirely.

In [ ]:
data_store = DATA_STORE_POOL.get_store('cci-zarr-store')
list(data_store.get_data_ids())

There are two sea ice datasets. We select the one about Sea ice Thickness for the Northern Hemisphere and proceed to show the contents of this dataset:

In [ ]:
seaice_descriptor=data_store.describe_data('ESACCI-SEAICE-L3C-SITHICK-SIRAL_CRYOSAT2-NH25KMEASE2-2010-2017-fv2.0.zarr')
display(seaice_descriptor)

Now let us open it. The parameter 'data_store_id' is not absolutely necessary, but it makes the opening a little faster. The parameter 'normalize' should be used so that the dataset is preprocessed in a way that it can be optimally used in Cate.

In [ ]:
# Open the Northern Hemisphere gridded dataset
seaice_th_ds=open_dataset(ds_id="ESACCI-SEAICE-L3C-SITHICK-SIRAL_CRYOSAT2-NH25KMEASE2-2010-2017-fv2.0.zarr",
                          data_store_id='cci-zarr-store',
                          normalize=True)

## Freeboard and (approximate) sea ice thickness

To first order, one can apply the Archimedes principle to convert the freeboard height to the thickness of the sea ice. To do this one could take the variable `freeboard`, which has a correction for the less dense snow on top of the ice corrected for and convert it using the following formula, assuming a uniform ice and sea water density $\rho_{ice}$ and $\rho_{w}$ respectively.
![Freeboard versus ice thickness](media/cryosatfreeboard.gif)

$h_{seaice}=h_{freeboard}\left(\frac{\rho_{w}}{\rho_{w}-\rho_{ice}}\right)$

So using reasonable values for the densities, we can compute a factor which gives us an idea how the freeboard height relates to the sea ice thickness. It turns out that only about 1/8th of the actual sea ice thickness is visible above the waterline as freeboard:

In [ ]:
rho_ice=900 # ice density kg/m^3
rho_w=1027 # sea water density kg/m^3

freeb2seaice=rho_w/(rho_w-rho_ice)
print (f"Approximate freeboard to sea ice thickness conversion factor {freeb2seaice}")

The dataset, which we opened above, has a variable called `sea_ice_thickness` which uses better estimates of the ice and sea water densities, and also incorporates the density of the snow layer which is sitting on top of the sea ice. So we can take that variable and make an animation.

## Create an animation of the arctic sea ice thickness

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import animation,cm
from IPython.display import HTML
import numpy as np
from copy import copy
# Make sure that the initial iconcentrationmage is not shown inline in the notebook
%matplotlib

# We also want to visualize the chang over time so we create an animation with the python module matplotlib
# You can find more information on matplotlib animations here: https://matplotlib.org/stable/api/animation_api.html

ith=0
fig, ax = plt.subplots()
fig.set_size_inches(8,8)

cmSea=copy(cm.get_cmap("viridis"))
cmSea.set_bad('grey')

#let's strip off the irrelevant edges which contain Nan's
xrng=np.arange(120,300)
yrng=np.arange(120,300)

im=ax.imshow(seaice_th_ds.sea_ice_thickness.isel(time=ith,xc=xrng,yc=yrng).data,cmap=cmSea)
cbar=ax.figure.colorbar(im,ax=ax)
cbar.ax.set_ylabel("meter")

nframes=seaice_th_ds.dims["time"]

def initSeaice():
    ax.set_title("Sea ice thickness %s"%np.datetime_as_string(seaice_th_ds.time[ith],unit="D"))    
    return (im,)

def animSeaIce(i):
    print(f"Animating frame {i} of {nframes - 1}", end="\r")
    ax.set_title("Sea ice thickness %s"%np.datetime_as_string(seaice_th_ds.time[i],unit="D"))
    im.set_array(seaice_th_ds.sea_ice_thickness.isel(time=i,xc=xrng,yc=yrng).data)
    return (im,)



anim = animation.FuncAnimation(fig, animSeaIce, init_func=initSeaice,
                               frames=nframes, interval=100, 
                               blit=True)

# The to_jshtml call creates some html code and javascript which allows us to control the animation
HTML(anim.to_jshtml())


## Conversion of sea ice thickness and sea ice concentration to sea ice volume

Upon a closer inspection of the dataset, one can also find a sea ice concentration variable, $C_{seaice}$, which has been interpolated on the same grid. This allows us to compute sea ice volume variations, $V_{seaice}$ over time by integrating over the valid data points of the Arctic:

$V_{seaice} (t)=\int_{Arctic} C_{seaice}(\theta,\lambda,t) h_{seaice}(\theta,\lambda,t) \sin \theta d\theta d\lambda$

We have the advantage that the data is already supplied as a Lambert azimuthal equal-area projection, so we can assign the same area, $dA$, to each 25kmx25km grid cell and simply convert the integral to a sum.

$V_{seaice} (t)\approx \sum_{i,j} C_{seaice}(i,j,t) h_{seaice}(i,j,t) dA$



In [ ]:
dA=25**2 # cell area in km^2
#note the conversion factor 1e-5 accounts for converting meter to km and sea ice concentration in percent to a fraction 
conv=1e-5
tmp=conv*seaice_th_ds.sea_ice_thickness*seaice_th_ds.sea_ice_concentration 
Vseaice=dA*tmp.sum(dim=['xc','yc'],skipna=True)


In [ ]:
%matplotlib inline 
fig, ax = plt.subplots()
fig.set_size_inches(8,8)
Vseaice.plot(ax=ax,marker='o')
ax.set_title("Arctic sea ice volume (for dataset coverage only)")
ax.set_ylabel("km^3")

# Questions

1. Try plotting the variables `freeboard`, `radar_freeboard`. How much do they differ? What is causing the difference?
2. To compare the behavior among the different seasons, try creating a plot which has the months as x-axis. Hint: you can use [xarray's groupby function](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.groupby.html) to sort the dataset.
3. From the animation it looks like the ice is drifting over time. Try to create so called [Hovmöller plots](https://cate.readthedocs.io/en/latest/use_cases.html?highlight=hovm%C3%B6ller#analysis-of-equatorial-aerosol-and-cloud-features-using-hovmoller-diagrams) of certain x,y sections (2D plot with the spatial coordinate on the x axis and the time axis on the y axis colored according to the sea ice thickness). Can you spot the sea ice drift in those plots?
